In [87]:
# code from https://github.com/sravanroy/RecommendationSystem-Word2vec/blob/master/word2vec.ipynb

In [80]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from gensim.models import Word2Vec 
import matplotlib.pyplot as plt
%matplotlib inline

import warnings;
warnings.filterwarnings('ignore')

In [82]:
df = pd.read_excel('Online Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


# Data Preprocessing

In [84]:
# check for missing values
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [85]:
# remove missing values
df.dropna(inplace=True)

#convert stock_code to string type since it's a unique combinations of numbers and letters
df['StockCode']= df['StockCode'].astype(str)

# Train and Validation Split

In [86]:
customers = df['CustomerID'].unique().tolist()

# shuffle customer ID's
random.shuffle(customers)

# extract 90% of customer ID's
customers_train = [customers[i] for i in range(round(0.9*len(customers)))]

# split data into train and validation set
train_df = df[df['CustomerID'].isin(customers_train)]
validation_df = df[~df['CustomerID'].isin(customers_train)]

# Create sequences of purchases made by the customers

In [88]:
# list to capture purchase history of the customers
purchases_train = []

# populate the list with the product codes
for i in tqdm(customers_train):
    temp = train_df[train_df["CustomerID"] == i]["StockCode"].tolist()
    purchases_train.append(temp)

100%|██████████| 3935/3935 [00:01<00:00, 2212.28it/s]


In [89]:
# list to capture purchase history of the customers
purchases_val = []

# populate the list with the product codes
for i in tqdm(validation_df['CustomerID'].unique()):
    temp = validation_df[validation_df["CustomerID"] == i]["StockCode"].tolist()
    purchases_val.append(temp)

100%|██████████| 437/437 [00:00<00:00, 4444.97it/s]


# Build Word2Vec Embeddings for products

In [100]:
# train word2vec model
model = Word2Vec(window = 10, sg = 1, hs = 0,
                 negative = 10, # for negative sampling
                 alpha=0.03, min_alpha=0.0007,
                 seed = 14)

model.build_vocab(purchases_train, progress_per=200)

model.train(purchases_train, total_examples = model.corpus_count, 
            epochs=10, report_delay=1)

(3635960, 3672140)

In [101]:
model.init_sims(replace=True)
print(model)

Word2Vec<vocab=3186, vector_size=100, alpha=0.03>


# Extract the vectors for all words

In [109]:
from gensim.models import keyedvectors

# extract all vectors
X = model.wv
X

# Visualize word2vec embeddings by reducing dimensions using tSNE

In [104]:
from sklearn.manifold import TSNE

In [112]:
def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [113]:
tsne_plot(model)

AttributeError: The vocab attribute was removed from KeyedVector in Gensim 4.0.0.
Use KeyedVector's .key_to_index dict, .index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4

# Product Reccomendations

### Creating a product ID-Description dictionary

In [114]:
products = train_df[["StockCode", "Description"]]

# remove duplicates
products.drop_duplicates(inplace=True, subset='StockCode', keep="last")

# create product-ID and product-description dictionary
products_dict = products.groupby('StockCode')['Description'].apply(list).to_dict()

### Function to return top 5 products based on a purchase

In [117]:
def similar_products(v, n = 5):
    
    # extract most similar products for the input vector
    ms = model.similar_by_vector(v, topn= n+1)[1:]
    
    # extract name and similarity score of the similar products
    new_ms = []
    for j in ms:
        pair = (products_dict[j[0]][0], j[1])
        new_ms.append(pair)
        
    return new_ms

### Test recommendations for a product 16218 ("CARTOON PENCIL SHARPENER")

In [118]:
similar_products(model['16218'])

TypeError: 'Word2Vec' object is not subscriptable

### Function to return the mean of vectors for recommendations based on multiple purchases

In [119]:
def aggregate_vectors(products):
    product_vec = []
    for i in products:
        try:
            product_vec.append(model[i])
        except KeyError:
            continue
        
    return np.mean(product_vec, axis=0)

### Using the validation data to get recommendations based on entire purchase history of a customer

In [120]:
# pick a customer in validation data and get the mean vector of his purchases

similar_products(aggregate_vectors(purchases_val[8]))

TypeError: 'Word2Vec' object is not subscriptable

In [121]:
# recommendations based on the last 10 products purchased
similar_products(aggregate_vectors(purchases_val[8][-10:]))

TypeError: 'Word2Vec' object is not subscriptable

In [64]:
import numpy as np  
import matplotlib.pyplot as plt  
import pandas as pd  
from apyori import apriori

In [51]:
import pandas as pd

df = pd.read_excel('Online Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [52]:
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')

In [53]:
df.shape

(541909, 8)

In [54]:
df = df[df['Country']=='United Kingdom'][0:10000]

In [55]:
df.shape

(10000, 8)

In [56]:
df = df[['InvoiceNo', 'StockCode']]
df.head()

,InvoiceNo,StockCode
0,536365,85123A
1,536365,71053
2,536365,84406B
3,536365,84029G
4,536365,84029E


In [57]:
df = df[~df['InvoiceNo'].str.contains('C')]

In [58]:
df = df.pivot(columns='StockCode')

In [59]:
df

InvoiceNo                                                        \
StockCode     10002 10120 10125 10133 10135 11001 15034 15036 15039 16010   
0               NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
1               NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2               NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
3               NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4               NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
...             ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
10593           NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
10594           NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
10595           NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
10596           NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
10597           NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

           ...                                                                \
StockCode  ... 90214L 90214M 90214N 90214R 90214S 90214V 90214Y BANK CHARGES   
0          ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN          NaN   
1          ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN          NaN   
2          ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN          NaN   
3          ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN          NaN   
4          ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN          NaN   
...        ...    ...    ...    ...    ...    ...    ...    ...          ...   
10593      ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN          NaN   
10594      ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN          NaN   
10595      ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN          NaN   
10596      ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN          NaN   
10597      ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN          NaN   

                     
StockCode  DOT    M  
0          NaN  NaN  
1          NaN  NaN  
2          NaN  NaN  
3          NaN  NaN  
4          NaN  NaN  
...        ...  ...  
10593      NaN  NaN  
10594      NaN  NaN  
10595      NaN  NaN  
10596      NaN  NaN  
10597      NaN  NaN  

[9916 rows x 2012 columns]

In [60]:
df.values

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=object)

In [62]:
num_records = len(df)
print(num_records)

9916


In [63]:
records = []  
for i in range(0, num_records):  
    records.append([str(df.values[i,j]) for j in range(0, 20)])

In [75]:
records

[['nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan'],
 ['nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan'],
 ['nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan'],
 ['nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan'],
 ['nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan'],
 ['nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'na

In [68]:
support = 30/num_records
support

0.0030254134731746672

In [78]:
association_rules = apriori(records, min_support=0.0010, min_confidence=0.10, min_lift=1.1, min_length=1)
association_results = list(association_rules)  

In [79]:
print(len(association_results))

0


## Step 1: define the features

In [6]:
from sklearn.preprocessing import StandardScaler

# Extract only some subset of columns to reduce computation time 
X = df[['mpg', 'disp', 'hp', 'wt']].values

# Standardize the features so that no feature dominates the
# distance computations due to unit scale
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

KeyError: "None of [Index(['mpg', 'disp', 'hp', 'wt'], dtype='object')] are in the [columns]"